In [1]:
import torch
import torchvision
import cv2

/root/miniconda3/envs/ipynb/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
weights = torch.load("/home/akiirabino/video-anal/ipynb/model/market.pth")

In [3]:
model = torchvision.models.resnet101(weights=weights)

/root/miniconda3/envs/ipynb/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
img = cv2.resize(cv2.imread("/home/akiirabino/video-anal/ipynb/dataset-1/image.png"), (640,640))


In [5]:
import io
import onnx
inputs = torch.randn(1, 3,640, 640)
onnx_model = None
with io.BytesIO() as f:
    torch.onnx.export(
        model,
        inputs,
        f,
        operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,                
    )
    onnx_model = onnx.load_from_string(f.getvalue())

In [6]:
onnx.save_model(onnx_model, "market.onnx")

In [7]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [8]:
import onnxruntime
ort_sess = onnxruntime.InferenceSession("/home/akiirabino/video-anal/ipynb/model/market.onnx")


In [ ]:
input_name = ort_sess.get_inputs()[0].name